In [ ]:
!pip install mmcv
import pandas as pd
import os
import json
import pickle
import numpy as np
import cv2
import mmcv
from PIL import Image, ImageDraw

In [3]:
#given the pose list and the dataframe with candidate face positions
#get only the candidate's poses
def get_candidate_poses(poselist, fr_path):

    l = pd.read_pickle(fr_path)
    df = pd.DataFrame.from_dict(l, orient = "index", columns = ["x0", "y0", "x1", "y1"])

    candidate_pose_landmarks = []

    for i, frame in enumerate(poselist):

        candidate_in_frame = []

        for person_keypoints in frame:

            pose_face_x = person_keypoints[0:2][0]
            pose_face_y = person_keypoints[0:2][1]

            if i in df.index:
                in_x = df.loc[[i]]['x1'].item() >= pose_face_x >= df.loc[[i]]['x0'].item()
                in_y = df.loc[[i]]['y1'].item() >= pose_face_y >= df.loc[[i]]['y0'].item()

                #is the pose face the candidate?
                is_candidate = in_x & in_y

                if is_candidate:
                    candidate_in_frame = person_keypoints

        candidate_pose_landmarks.append(candidate_in_frame)

    return(candidate_pose_landmarks)


#
def fill_cand_array(cand_array, poselist):

    for i, row in enumerate(cand_array):

        if i == (len(cand_array)-1):
            break

        #print(i)
        #print(row)
        
        #If the current row is nan
        if all(np.isnan(cand_array[i])):
            #And the next row is not nan
            if all(np.isnan(cand_array[i+1])) == False:

                match_found = True
                d=0

                #backfill
                while (match_found == True) & (i-d>=0) & all(np.isnan(cand_array[i-d])):

                    match_found = False

                    for j in poselist[i-d]:

                        frame_distance = np.abs(np.nanmean(np.absolute(cand_array[i+1-d]-np.array(j))))

                        if frame_distance <= 5:
                            cand_array[i-d] = np.array(j)

                            match_found = True
                            d += 1

                        del frame_distance

            #And the previous row is not nan
            #forward fill
            #here we only need to do it once rather than also retroactively for the rows before
            #because we are iterating over all rows in a forward manner anyway
            elif all(np.isnan(cand_array[i-1])) == False:

                for j in poselist[i]:

                        frame_distance = np.abs(np.nanmean(np.absolute(cand_array[i-1]-np.array(j))))

                        if frame_distance <= 5:
                            cand_array[i] = np.array(j)

                        del frame_distance

    return(cand_array)


In [6]:
!cp /content/drive/MyDrive/projects/pose_detection/replication/video.pkl .

In [7]:
#----
posedir = "./"
frdir = "./"
#fr_out = "GOV_AK_BEGICH_HIS_EXAMPLE_cand_boxes.pkl"
#fr_path = frdir + fr_out

poselist_paths = os.listdir(posedir)
fr_results_files = os.listdir(frdir)

fr_results_names = [s.replace("_cand_boxes.pkl", "") for s in fr_results_files]

#Make a dictionary where the keys are the locations of the face recognition
#And the values are the locations of the pose detection results
ref_dict = {}
for i, name in enumerate(fr_results_names):
    if os.path.exists(posedir + name + ".pkl"):
        ref_dict[fr_results_files[i]] = posedir + name + ".pkl"

In [8]:
ref_dict

{'video_cand_boxes.pkl': './video.pkl'}

In [9]:
counter = 0
for k in ref_dict:

    #check if file has already been done, if so, skip
    output_path = './' + k.replace("_cand_boxes.pkl", "landmarks.csv")
    if os.path.exists(output_path)==False:

        print(ref_dict[k])
        #Load the poselist
        posel = pickle.load(open( ref_dict[k], "rb" ))
        #combine the poselist with the facial recognition to get only the candidate's poses
        fr_path = frdir + k
        candidate_poses = get_candidate_poses(posel, fr_path)
        #convert to pd dataframe and then np array
        df_candidate = pd.DataFrame(candidate_poses)
        cand_array = np.array(df_candidate)
        #fill out missing rows in the array if they PROBABLY belong to the candidate
        cand_array = fill_cand_array(cand_array, posel)

        #Convert the numpy to a pandas dataframe...
        colnamesx = ["x" + str(i) for i in range(25)]
        colnamesy = ["y" + str(i) for i in range(25)]
        colnames = [None]*(len(colnamesx)+len(colnamesy))
        colnames[::2] = colnamesx
        colnames[1::2] = colnamesy
        if cand_array.size == 0:
            cand_df = pd.DataFrame(columns = colnames)
        else:
            cand_df = pd.DataFrame(cand_array, columns = colnames)
        #... and save it
        cand_df.to_csv(output_path, index=False)
        print(counter)
        counter += 1



./video.pkl
0
